# Joining and Appending DataFrames in PySpark HW

Now it's time to test your knowledge and further engrain the concepts we touched on in the lectures. Let's go ahead and get started.




**As always let's start our Spark instance.**

In [1]:

import pyspark # only run after findspark.init()
from pyspark.sql import SparkSession
# May take awhile locally
spark = SparkSession.builder.appName("joins").getOrCreate()

cores = spark._jsc.sc().getExecutorMemoryStatus().keySet().size()
print("You are working with", cores, "core(s)")
spark

You are working with 1 core(s)


## Read in the database

Let cotinue working with our college courses dataframe to get some more insights and practice what we have learned!Let's read in the whole database using the loop function that we learned about in the lecture to automatically read in all the datasets from the uw-madision-courses folder (there are too many datasets to each one individually.

In [2]:
import os

path = "Datasets/uw-madison-courses/"

df_list = []
for filename in os.listdir(path):
    if filename.endswith(".csv"):
        filename_list = filename.split(".") #separate path from .csv
        df_name = filename_list[0]
        df = spark.read.csv(path+filename,inferSchema=True,header=True)
        df.name = df_name
        df_list.append(df_name)
        exec(df_name + ' = df')
        
# QA
print("Full list of dfs:")
print(df_list)

Full list of dfs:
['courses', 'course_offerings', 'grade_distributions', 'instructors', 'rooms', 'schedules', 'sections', 'subjects', 'subject_memberships', 'teachings']


Now check the contents of a few of the dataframses that were read in above.

In [3]:
rooms.toPandas()

,uuid,facility_code,room_code
0,04368a56-c959-3e4b-8b3d-f4cc3538fea5,OFF CAMPUS,null
1,2cc50da3-ef0e-3572-a557-ca44930a0688,0032,0249
2,ebbf62b4-2ac3-356b-b0fa-7897f4446a17,0032,B101
3,ed828265-475b-31b4-b9a8-daec2a600449,0032,0549
4,b277dc8e-bab1-3a12-bc17-48d4a364d297,0032,0349
...,...,...,...
1345,d3ace3fc-91f9-30fc-ae40-be2dba4187a7,0103,0012
1346,41be44ad-3ef7-3184-ab45-bff1b924eb49,0085,0448
1347,0a0ae4a0-2139-35e0-a927-c495a296d174,1245,0B32
1348,c48297dd-bcaa-31ef-933f-fd6c41601595,0482,1236


In [4]:
course_offerings.toPandas()

,uuid,course_uuid,term_code,name
0,344b3ebe-da7e-314c-83ed-9425269695fd,a3e3e1c3-543d-3bb5-ae65-5f2aec4ad1de,1092,Cooperative Education Prog
1,f718e6cd-33f0-3c14-a9a6-834d9c3610a8,a3e3e1c3-543d-3bb5-ae65-5f2aec4ad1de,1082,Cooperative Education Prog
2,ea3b717c-d66b-30dc-8b37-964d9688295f,a3e3e1c3-543d-3bb5-ae65-5f2aec4ad1de,1172,Cooperative Education Prog
3,075da420-5f49-3dd0-93df-13e3c152e1b1,a3e3e1c3-543d-3bb5-ae65-5f2aec4ad1de,1114,Cooperative Education Prog
4,2b4e216d-a728-3713-8c7c-19afffc6b2fd,a3e3e1c3-543d-3bb5-ae65-5f2aec4ad1de,1104,Cooperative Education Prog
...,...,...,...,...
81447,3573e335-e7be-33a6-b023-8704b3acc93b,4a7f3d9d-f2e5-3df2-8dee-2c2d02ed9098,1182,Conservation Planning
81448,5d437083-a1d0-38b8-aa70-ec3564da39a6,1220244c-2bbf-3169-8eb0-6a2da36f40f6,1182,null
81449,13278922-0726-3823-abda-dc14df03d8a6,516f637c-a244-3df0-ba3f-f11ce650543d,1182,Patient Ed. as Advocacy Strat.
81450,fd039fe4-32ec-31dd-a142-c7db54df784b,fbe0838f-9469-3d83-ad85-91ba5aabd1a3,1182,Multid. Appr. to Colon Cancer


## Recap: About this database

You will notice that there are several more tables in the uw-madision-courses folder than there are read in above. This so that you will have a chance to practice your own custom joins and learn about the relationships between a real database work. Sometimes we don't know how they are related and we need to figure it out! I'll save that for the HW :) 

Here is a look at some of the important variables we can use to join our tables:

 - course_offerings: uuid, course_uuid, term_code, name
 - instructors: id, name
 - schedules: uuid
 - sections: uuid, course_offering_uuid,room_uuid, schedule_uuid
 - teachings: instructor_id, section_uuid
 - courses: uuid
 - grade_distributions: course_offering_uuid,section_number
 - rooms: uuid, facility_code, room_code
 - subjects: code
 - subject_memberships: subject_code, course_offering_uuid
 
 **Source:** https://www.kaggle.com/Madgrades/uw-madison-courses
 
So alright, let's use this information to discover some insights from this data!

## 1a. Can you assign the room numbers to each section of each course?

Show only the rooms uuid, facility code, room number, term code and the name of the course from the course_offerings table.

In [39]:
rooms_new =rooms.withColumnRenamed('uuid', 'room_uuid')
joined_data1=rooms_new.join(sections , "room_uuid" , how='inner').select(['room_uuid','facility_code','room_code','course_offering_uuid'])
joined_data1.show()

+--------------------+-------------+---------+--------------------+
|           room_uuid|facility_code|room_code|course_offering_uuid|
+--------------------+-------------+---------+--------------------+
|04368a56-c959-3e4...|   OFF CAMPUS|     null|ea3b717c-d66b-30d...|
|04368a56-c959-3e4...|   OFF CAMPUS|     null|ea3b717c-d66b-30d...|
|04368a56-c959-3e4...|   OFF CAMPUS|     null|075da420-5f49-3dd...|
|04368a56-c959-3e4...|   OFF CAMPUS|     null|2b4e216d-a728-371...|
|04368a56-c959-3e4...|   OFF CAMPUS|     null|ff511882-5eab-3e7...|
|04368a56-c959-3e4...|   OFF CAMPUS|     null|ff511882-5eab-3e7...|
|04368a56-c959-3e4...|   OFF CAMPUS|     null|b3e26604-fe6e-30d...|
|04368a56-c959-3e4...|   OFF CAMPUS|     null|e100d196-5e82-32e...|
|04368a56-c959-3e4...|   OFF CAMPUS|     null|e100d196-5e82-32e...|
|04368a56-c959-3e4...|   OFF CAMPUS|     null|76b9c458-d3c2-38c...|
|04368a56-c959-3e4...|   OFF CAMPUS|     null|76b9c458-d3c2-38c...|
|04368a56-c959-3e4...|   OFF CAMPUS|     null|76

In [41]:
joined_data2=joined_data1.join(course_offerings , course_offerings.uuid ==joined_data1.course_offering_uuid  , how='inner').select(['room_uuid','facility_code','room_code','term_code','name'])
joined_data2.show()

+--------------------+-------------+---------+---------+--------------------+
|           room_uuid|facility_code|room_code|term_code|                name|
+--------------------+-------------+---------+---------+--------------------+
|04368a56-c959-3e4...|   OFF CAMPUS|     null|     1172|Cooperative Educa...|
|04368a56-c959-3e4...|   OFF CAMPUS|     null|     1172|Cooperative Educa...|
|04368a56-c959-3e4...|   OFF CAMPUS|     null|     1114|Cooperative Educa...|
|04368a56-c959-3e4...|   OFF CAMPUS|     null|     1104|Cooperative Educa...|
|04368a56-c959-3e4...|   OFF CAMPUS|     null|     1134|Cooperative Educa...|
|04368a56-c959-3e4...|   OFF CAMPUS|     null|     1134|Cooperative Educa...|
|04368a56-c959-3e4...|   OFF CAMPUS|     null|     1084|Cooperative Educa...|
|04368a56-c959-3e4...|   OFF CAMPUS|     null|     1154|Cooperative Educa...|
|04368a56-c959-3e4...|   OFF CAMPUS|     null|     1154|Cooperative Educa...|
|04368a56-c959-3e4...|   OFF CAMPUS|     null|     1162|Cooperat

## 1b. Now show same output as above but for only facility number 0469 (facility_code)

In [44]:
# joined_data=rooms.filter(rooms.facility_code == '0469').join(course_offerings, "uuid", how='left')\
#                                                         .select(['uuid','facility_code','room_code','term_code','name'])
# joined_data.show(10)
# joined_data.where(joined_data['facility_code'] == '0469').show(10)

In [42]:
joined_data1=rooms_new.filter(rooms.facility_code == '0469').join(sections , "room_uuid" , how='inner').select(['room_uuid','facility_code','room_code','course_offering_uuid'])
joined_data1.show()

+--------------------+-------------+---------+--------------------+
|           room_uuid|facility_code|room_code|course_offering_uuid|
+--------------------+-------------+---------+--------------------+
|9759cb5f-a7d3-3d0...|         0469|     2441|bd5baa02-e0ca-39d...|
|9759cb5f-a7d3-3d0...|         0469|     2441|2101e24a-f58f-33d...|
|6af80b0b-b3e3-370...|         0469|     4411|821927cc-c5eb-38c...|
|9759cb5f-a7d3-3d0...|         0469|     2441|a35e69d8-4355-3bf...|
|9759cb5f-a7d3-3d0...|         0469|     2441|3e551eb9-a026-3eb...|
|50322d30-dd8f-3c6...|         0469|     2511|19968d09-a08d-3e0...|
|9759cb5f-a7d3-3d0...|         0469|     2441|42440324-78e9-3db...|
|9759cb5f-a7d3-3d0...|         0469|     2441|c395095e-d2ec-38f...|
|9759cb5f-a7d3-3d0...|         0469|     2441|7bcdf8aa-f943-38f...|
|50322d30-dd8f-3c6...|         0469|     2511|011b2eae-6f5e-3e2...|
|9759cb5f-a7d3-3d0...|         0469|     2441|9bba8c77-b564-301...|
|9759cb5f-a7d3-3d0...|         0469|     2441|aa

In [43]:
joined_data2=joined_data1.join(course_offerings , course_offerings.uuid ==joined_data1.course_offering_uuid  , how='inner').select(['room_uuid','facility_code','room_code','term_code','name'])
joined_data2.show()

+--------------------+-------------+---------+---------+--------------------+
|           room_uuid|facility_code|room_code|term_code|                name|
+--------------------+-------------+---------+---------+--------------------+
|9759cb5f-a7d3-3d0...|         0469|     2441|     1152|  Fundamentals-Flute|
|9759cb5f-a7d3-3d0...|         0469|     2441|     1092|  Fundamentals-Flute|
|6af80b0b-b3e3-370...|         0469|     4411|     1172|  Fundamentals-Flute|
|9759cb5f-a7d3-3d0...|         0469|     2441|     1162|  Fundamentals-Flute|
|9759cb5f-a7d3-3d0...|         0469|     2441|     1132|  Fundamentals-Flute|
|50322d30-dd8f-3c6...|         0469|     2511|     1072|  Fundamentals-Flute|
|9759cb5f-a7d3-3d0...|         0469|     2441|     1142|  Fundamentals-Flute|
|9759cb5f-a7d3-3d0...|         0469|     2441|     1112|  Fundamentals-Flute|
|9759cb5f-a7d3-3d0...|         0469|     2441|     1102|  Fundamentals-Flute|
|50322d30-dd8f-3c6...|         0469|     2511|     1082|  Fundam

In [29]:
# rooms.select('uuid','facility_code').filter(rooms['facility_code'] == '0469').show()

## 2. Count how many sections are offered for each subject for each facility

*Note: this will involve a groupby*

In [45]:
sections_new =sections.withColumnRenamed('uuid', 'section_uuid')
joined_data1 = rooms_new.join(sections_new , "room_uuid" , how='inner')\
                                        .select(['section_uuid','facility_code','course_offering_uuid'])
joined_data1.show()

+--------------------+-------------+--------------------+
|        section_uuid|facility_code|course_offering_uuid|
+--------------------+-------------+--------------------+
|9395dc21-15d1-3fa...|   OFF CAMPUS|ea3b717c-d66b-30d...|
|da41b0aa-2b81-378...|   OFF CAMPUS|ea3b717c-d66b-30d...|
|51c4dc00-1fc7-3c7...|   OFF CAMPUS|075da420-5f49-3dd...|
|8fc362a6-d94e-3ad...|   OFF CAMPUS|2b4e216d-a728-371...|
|b21564d2-2bc4-3b8...|   OFF CAMPUS|ff511882-5eab-3e7...|
|8853d23e-64b1-3cf...|   OFF CAMPUS|ff511882-5eab-3e7...|
|dfb55b25-ac7a-366...|   OFF CAMPUS|b3e26604-fe6e-30d...|
|56ef2b0f-0776-309...|   OFF CAMPUS|e100d196-5e82-32e...|
|ad67e659-fbf4-32a...|   OFF CAMPUS|e100d196-5e82-32e...|
|c72fe508-e5cb-311...|   OFF CAMPUS|76b9c458-d3c2-38c...|
|d6718082-bd04-378...|   OFF CAMPUS|76b9c458-d3c2-38c...|
|6c32e394-bbae-3c6...|   OFF CAMPUS|76b9c458-d3c2-38c...|
|2c983272-9f46-3d9...|   OFF CAMPUS|c94b0026-46e5-318...|
|d53b52c5-7538-3e4...|   OFF CAMPUS|c94b0026-46e5-318...|
|9465796d-7f08

In [46]:
joined_data2=joined_data1.join(subject_memberships , ["course_offering_uuid"] , how='inner')\
                                        .select(['section_uuid','facility_code','course_offering_uuid','subject_code'])
joined_data2.show()

+--------------------+-------------+--------------------+------------+
|        section_uuid|facility_code|course_offering_uuid|subject_code|
+--------------------+-------------+--------------------+------------+
|9395dc21-15d1-3fa...|   OFF CAMPUS|ea3b717c-d66b-30d...|         240|
|9395dc21-15d1-3fa...|   OFF CAMPUS|ea3b717c-d66b-30d...|         418|
|9395dc21-15d1-3fa...|   OFF CAMPUS|ea3b717c-d66b-30d...|         490|
|9395dc21-15d1-3fa...|   OFF CAMPUS|ea3b717c-d66b-30d...|         636|
|9395dc21-15d1-3fa...|   OFF CAMPUS|ea3b717c-d66b-30d...|         207|
|9395dc21-15d1-3fa...|   OFF CAMPUS|ea3b717c-d66b-30d...|         612|
|9395dc21-15d1-3fa...|   OFF CAMPUS|ea3b717c-d66b-30d...|         346|
|9395dc21-15d1-3fa...|   OFF CAMPUS|ea3b717c-d66b-30d...|         684|
|9395dc21-15d1-3fa...|   OFF CAMPUS|ea3b717c-d66b-30d...|         320|
|9395dc21-15d1-3fa...|   OFF CAMPUS|ea3b717c-d66b-30d...|         220|
|da41b0aa-2b81-378...|   OFF CAMPUS|ea3b717c-d66b-30d...|         240|
|da41b

In [54]:
joined_data2.groupBy(["subject_code","facility_code"]).count().toPandas()

,subject_code,facility_code,count
0,448,0046,193
1,448,0050,36
2,900,0060,14
3,208,0400,10
4,351,0055,2
...,...,...,...
3961,244,0402,1
3962,490,0054,1
3963,240,0485,1
3964,900,0544,2


## 3. What are the hardest classes?

Let's see if we can figure out which classes are the hardest by seeing how many students failed. Note that you will first need to aggregate the grades table by the course uuid to include all sections. Show the name of the course as well that you will need to get from the course_offering table.

In [88]:
grouped= grade_distributions.select('course_offering_uuid','f_count').filter(grade_distributions.f_count >0)\
                                        .groupBy('course_offering_uuid').agg(sum(grade_distributions.f_count).alias('total_F'))
grouped.show()

+--------------------+-------+
|course_offering_uuid|total_F|
+--------------------+-------+
|cfeba94d-8e0e-320...|      7|
|ceabe145-78e5-33c...|      4|
|3b91b41e-1888-39c...|      1|
|2e439330-d5b5-3a3...|      1|
|951bb4bd-2055-355...|      4|
|0456a49e-b717-364...|      4|
|f8583533-03d8-39a...|      4|
|de01255d-3488-3d2...|      2|
|9160b322-6104-34e...|      7|
|2f3ad380-d58d-37b...|     24|
|a0eb0cec-f81f-3e7...|      3|
|4d0a3374-abad-3a3...|      1|
|740fd472-59e4-3f3...|      1|
|c7d3c9fd-dbc2-34b...|      5|
|93735d84-97f6-324...|      4|
|06f15d4f-c8c7-3c8...|      1|
|00167c90-18b1-3a6...|      2|
|6e312a9e-dab2-350...|      1|
|2a666e1a-e17b-3e1...|      1|
|057515db-64b3-361...|      4|
+--------------------+-------+
only showing top 20 rows



In [89]:
joined_data= grouped.join(course_offerings , course_offerings.uuid == grade_distributions.course_offering_uuid , how='inner')\
                                            .select(['course_offering_uuid','total_F','name'])
joined_data.show()


+--------------------+-------+--------------------+
|course_offering_uuid|total_F|                name|
+--------------------+-------+--------------------+
|b3e26604-fe6e-30d...|      1|Cooperative Educa...|
|e6fe4acb-9375-3b4...|      1|Cooperative Educa...|
|04a206fe-cf6a-3c7...|      1|Cooperative Educa...|
|f84c7f22-aa01-3d7...|      1|Cooperative Educa...|
|3c59e623-7e61-343...|      3|Workshop in Dance...|
|a37b96cc-0586-3ea...|      3|Workshop in Dance...|
|dd3caaea-05bf-383...|      1|Workshop in Dance...|
|ea44cb23-33f5-354...|      1|Workshop in Dance...|
|1e9f79ac-0850-3ad...|      1|Workshop in Dance...|
|a97d123e-7005-395...|      2|Workshop in Dance...|
|a254463d-8440-334...|      1|Workshop in Dance...|
|da86adc6-3c59-3a0...|      1|    Ballroom Dance I|
|5b62040e-c0d8-355...|      1|    Ballroom Dance I|
|81f845ea-d594-3e0...|      1|    Ballroom Dance I|
|ac6ec56e-ad7d-3d1...|      1|      Modern Dance I|
|1f151924-65cf-32d...|      1|      Modern Dance I|
|50e08207-c9

In [105]:
joined_data.select('name','total_F').orderBy(joined_data.total_F.desc()).show(1,False)

+--------------------------+-------+
|name                      |total_F|
+--------------------------+-------+
|Calc--Functns of Variables|72     |
+--------------------------+-------+
only showing top 1 row



## Challenge Question: Automating data entry errors

We see in the dataframe below that there are several typos of various animal names. If this was a large database of several millions of records, correcting these errors would be way too labor intensive. How can we automate correcting these errors?

*Hint: Leven...*

In [2]:
values = [('Monkey',10),('Monkay',36),('Mnky',123), \
          ('Elephant',48),('Elefant',16),('Ellafant',1), \
          ('Hippopotamus',48),('Hipopotamus',16),('Hippo',1)]
zoo = spark.createDataFrame(values,['Animal','age'])
zoo.show()

+------------+---+
|      Animal|age|
+------------+---+
|      Monkey| 10|
|      Monkay| 36|
|        Mnky|123|
|    Elephant| 48|
|     Elefant| 16|
|    Ellafant|  1|
|Hippopotamus| 48|
| Hipopotamus| 16|
|       Hippo|  1|
+------------+---+



In [5]:
import pyspark.sql.functions as F
import pyspark.sql.types as T
df = spark.createDataFrame(data=[["Monkey"], ["Elephant"], ["Hippopotamus"]], schema=['Animal_1'])
df_cross= zoo.crossJoin(df)
df_cross=df_cross.withColumn("levenshtein", F.levenshtein(F.col("Animal_1"), F.col("Animal")))
df_cross.filter("levenshtein <=5").show()
# df_cross.show(50)

+------------+---+------------+-----------+
|      Animal|age|    Animal_1|levenshtein|
+------------+---+------------+-----------+
|      Monkey| 10|      Monkey|          0|
|      Monkay| 36|      Monkey|          1|
|        Mnky|123|      Monkey|          2|
|    Elephant| 48|    Elephant|          0|
|     Elefant| 16|    Elephant|          2|
|    Ellafant|  1|    Elephant|          3|
|Hippopotamus| 48|Hippopotamus|          0|
| Hipopotamus| 16|Hippopotamus|          1|
+------------+---+------------+-----------+



### Great job! 